<a href="https://colab.research.google.com/github/ClementineDomine/NeuralPlayground/blob/comparison_board/colab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeuralPlayground Introduction
## If you using colab, install requirements 

In [ ]:
!git clone --branch comparison_board https://github.com/ClementineDomine/NeuralPlayground

In [ ]:
!pip install -r NeuralPlayground/requirements.txt

In [ ]:
!pip install -e NeuralPlayground
exit()

In [ ]:
import neuralplayground